In [1]:
import os
import sys
import tensorflow as tf

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from ml.models.three_d import voxnet

Using TensorFlow backend.
/home/amy/elvo-analysis/ml/models/three_d/voxnet.py:83: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(input_shape=(1, 32, 32..., kernel_size=(5, 5, 5), filters=32, strides=(2, 2, 2), padding="valid", data_format="channels_first", kernel_initializer="normal", kernel_regularizer=<keras.reg..., bias_regularizer=<keras.reg...)`
  b_regularizer=l2(0.001),


DEBUG:root:Layer1:Conv3D shape=(None, 32, 14, 14, 14)
DEBUG:root:Layer3:Conv3D shape=(None, 32, 12, 12, 12)
DEBUG:root:Layer4:MaxPool3D shape=(None, 32, 6, 6, 6)
DEBUG:root:Layer5:Flatten shape=(None, 6912)
DEBUG:root:Layer6:Dense shape=(None, 128)
DEBUG:root:Layer8:Dense shape=(None, 2)
INFO:root:Model compiled!


/home/amy/elvo-analysis/venv/lib/python3.6/site-packages/keras/activations.py:186: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))
/home/amy/elvo-analysis/ml/models/three_d/voxnet.py:93: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.3)`
  self._mdl.add(Dropout(p=0.3))
/home/amy/elvo-analysis/ml/models/three_d/voxnet.py:105: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(kernel_size=(3, 3, 3), filters=32, strides=(1, 1, 1), padding="valid", data_format="channels_first", kernel_initializer="normal", kernel_regularizer=<keras.reg..., bias_regularizer=<keras.reg...)`
  b_regularizer=l2(0.001),
/home/amy/elvo-analysis/ml/models/three_d/voxnet.py:117: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(2, 2, 2), str

In [2]:
import os
import csv
import numpy as np
import random
from keras.preprocessing.image import ImageDataGenerator
from google.cloud import storage

BLACKLIST = []


class ChunkGenerator(object):
    def __init__(self, dims=(32, 32, 32), batch_size=16,
                 shuffle=True,
                 validation=False,
                 test=False, split_test=False,
                 split=0.2):
        self.dims = dims
        self.batch_size = batch_size
        self.validation = validation

        self.datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.1,
            height_shift_range=0.1,
            zoom_range=[1.0, 1.1],
        )

        # Access Google Cloud Storage
        gcs_client = storage.Client.from_service_account_json(
            '/home/amy/credentials/client_secret.json'
        )
        bucket = gcs_client.get_bucket('elvos')

        # Get label data from Google Cloud Storage
        blob = storage.Blob('augmented_annotated_labels.csv', bucket)
        blob.download_to_filename('/home/amy/elvo-analysis/augmented_annotated_labels.csv')
        prelim_label_data = {}
        with open('/home/amy/elvo-analysis/augmented_annotated_labels.csv', 'r') as pos_file:
            reader = csv.reader(pos_file, delimiter=',')
            for row in reader:
                if row[1] != 'Unnamed: 0':
                    prelim_label_data[row[1]] = int(row[2])

        # Get 12096 random negatives from the label data to feed into our generator
        negative_counter = 0
        negative_label_data = {}
        print("getting 12096 random negative labels")
        while negative_counter < 12000:
            id_, label = random.choice(list(prelim_label_data.items()))
            if label == 0:
                negative_label_data[id_] = label
                del prelim_label_data[id_]
                negative_counter += 1
        
        # Get all of the positives from the label data
        label_data = {}
        print('getting positive labels')
        for id_, label in list(prelim_label_data.items()):
            if label == 1:
                label_data[id_] = label
        
        print("negative label data length:" + str(len(negative_label_data)))
        print("positive label data length:" + str(len(label_data)))
        
        # Put pos and neg together into one dictionary
        label_data.update(negative_label_data)

        # Get positive chunks
        print('getting positive chunks')
        pos_blobs = bucket.list_blobs(prefix='chunk_data/normal/positive')
        files = []
        for blob in pos_blobs:
            file = blob.name

            # Check blacklist
            blacklisted = False
            for each in BLACKLIST:
                if each in file:
                    blacklisted = True
                    
            file_id = blob.name.split('/')[-1]
            file_id = file_id.split('.')[0]

            if not blacklisted:
                # Add all data augmentation methods
                files.append({
                    "name": file,
                })

        print("getting negative chunks")
        # Get negative chunks that were chosen to be in the labels
        neg_blobs = bucket.list_blobs(prefix='chunk_data/normal/negative')
        for blob in neg_blobs:
            file = blob.name

            # Check blacklist
            blacklisted = False
            for each in BLACKLIST:
                if each in file:
                    blacklisted = True

            file_id = blob.name.split('/')[-1]
            file_id = file_id.split('.')[0]

            if file_id in negative_label_data and not blacklisted
                and :
                files.append({
                    "name": file,
                })

        # convert labels from dict to np array
        print('converting labels from dict to np array')
        labels = np.zeros(len(files))
        for i, file in enumerate(files):
            filename = file['name']
            filename = filename.split('/')[-1]
            filename = filename.split('.')[0]
            labels[i] = int(label_data[filename])

        # Take into account shuffling
        if shuffle:
            tmp = list(zip(files, labels))
            random.shuffle(tmp)
            files, labels = zip(*tmp)
            labels = np.array(labels)

        # Split based on validation
        print('splitting based on validation split')
        if validation:
            if split_test:
                files = files[:int(len(files) * split / 2)]
                labels = labels[:int(len(labels) * split / 2)]
            else:
                files = files[:int(len(files) * split)]
                labels = labels[:int(len(labels) * split)]
        elif test:
            if split_test:
                files = files[int(len(files) * split / 2):
                              int(len(files) * split)]
                labels = labels[int(len(labels) * split / 2):
                                int(len(labels) * split)]
            else:
                raise ValueError('must set split_test to True if test')
        else:
            files = files[int(len(files) * split):]
            labels = labels[int(len(labels) * split):]
        print(np.shape(files))
        print(np.shape(labels))
        print("Negatives: {}".format(np.count_nonzero(labels == 0)))
        print("Positives: {}".format(np.count_nonzero(labels)))
        self.files = files
        self.labels = labels
        self.bucket = bucket

    def generate(self):
        steps = self.get_steps_per_epoch()
        # print(steps)
        while True:
            for i in range(steps):
                # print(i)
                # print("D")
                x, y = self.__data_generation(i)
                yield x, y

    def get_steps_per_epoch(self):
        return len(self.files) // self.batch_size

    def __data_generation(self, i):
        bsz = self.batch_size
        files = self.files[i * bsz:(i + 1) * bsz]
        labels = self.labels[i * bsz:(i + 1) * bsz]
        images = []

        # for improperly formatted images
        incorrect_shape_imgs = 0
        
        # Download files to tmp/npy/
        for i, file in enumerate(files):
            file_id = file['name'].split('/')[-1]
            file_id = file_id.split('.')[0]
            print(file_id)
            blob = self.bucket.get_blob(file['name'])
            blob.download_to_filename('/home/amy/data/chunks/{}.npy'
                                      .format(file_id))
            img = np.load('/home/amy/data/chunks/{}.npy'.format(file_id))
            print(np.shape(img))
            if np.shape(img)==(32, 32, 32):
                img = np.expand_dims(img, axis=0)
                images.append(img)
                print(np.shape(img))
            else:
                incorrect_shape_imgs += 1
                print("adding image to incorrect_shape_imgs")
                
            print("images shape: " + str(len(images)))

        print("final images shape: " + str(np.shape(images)))
        print("number incorrectly formatted images: " 
                  + str(incorrect_shape_imgs))
        
        images = np.array(images)
        
        print("Loaded entire batch.")
        print(labels)
        return images, labels



In [3]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [4]:
train_gen = ChunkGenerator()
val_gen = ChunkGenerator(validation=True)

DEBUG:urllib3.util.retry:Converted retries value: 3 -> Retry(total=3, connect=None, read=None, redirect=None, status=None)
DEBUG:google.auth.transport.requests:Making request: POST https://accounts.google.com/o/oauth2/token
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): accounts.google.com:443
DEBUG:urllib3.connectionpool:https://accounts.google.com:443 "POST /o/oauth2/token HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.googleapis.com:443
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos?projection=noAcl HTTP/1.1" 200 338
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /download/storage/v1/b/elvos/o/augmented_annotated_labels.csv?alt=media HTTP/1.1" 200 11956989
getting 12096 random negative labels
getting positive labels
negative label data length:12000
positive label data length:12675
getting positive chunks
DEBUG:urllib3.connectionpool:Resetting dropped connection: www.googl

DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjFjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS8yQlhDSk9RVkFZQ04wV0ZENzgubnB5&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861354
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS8yRVU5Q0xUR1BJNFY2WFVEMzI0Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861403
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS8yS01LWFIyRzFCTEQwQzJHMTI2Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861256
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS8yVTk1SE5JQklVV1RER0Y2MzYyLm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861591
DEBUG:ur

DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS82UVhTNTM1SEdDVDNRVTdaMjgxLm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861409
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS82U0ZGTFdBRDdUQ0NUVEhDMTAyLm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861469
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS82VUk1MkNGTVdIUjZVSDIyMjA5Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861540
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjBjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS82WTJIWVkyNDhFMFQ2SExPNS5ucHk%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861220
DEBUG:

DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9BU0QyVVJOS0ZSTjNaU0ZMNjE1Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861480
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9BVUdYVDVQSTRWWTYyTTZMMjg0Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 862023
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjFjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9BWFdGN1lDMU00SE00NkQzODkubnB5&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861342
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9CM1VaTVk0S0dGSVhQOEVVNDIxLm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861908
DEBUG:ur

DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9ESkdLREMzMkJFNU5LRUxYMTM4Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861462
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjFjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9ETDlMMlNOUzZVODBESTFBNTAubnB5&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861435
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9ETVlQTk9FWjlJV0dBOEFWMjY1Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861396
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9EUFNHNzVMQ0JaVkJUUlBPMTI3Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861438
DEBUG:ur

...

DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9OSUVENVVUT1oySk9GQ1dUNDM4Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861241
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9OSk5GVlpNU0dCR002SDVZNDM2Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861210
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9OSlpOMlROWFVVRlJaM0daMjE1Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861330
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9OTFhPU1VYT1paSFBGUVFYMTg1Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861141


DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9QVk1YRDNSWlNIRVFUWVczNjE4Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861272
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9QWFVYUk1JOFE0VE9GM0ZLMjk2Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861404
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9QWVpQN1dRNFFSSDQ2RUZTMjM4Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861275
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9RN1RaN0Q5TURGTk5UVUlFMjI0Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861350


DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9TRVYyR0FFRk5KQkxYM1pKNDQzLm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861756
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjNjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9TS0JJVDU3Q1BBS0FHUVZCMTIyOS5ucHk%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 862976
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjNjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9TTEk0NjdYNTdRRzhKTFhYMTkzNy5ucHk%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861654
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjFjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9TTUdXTURZVFlSOFpCM0Y1MTQubnB5&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861198
DEBUG:urllib

DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9VVUtTWVhCWVFRWkFUQlA1MTIzLm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861502
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjFjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9VWkZBTTQ1S1JJM0M3TUdCNjkubnB5&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861521
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9WMlRVWUxXWVFHRlVKT1hOMjk0Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861820
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9WQ0cyQUo0NFZJQU5ZQjEwOTQxLm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 862323
DEBUG:ur

DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9ZNExLRDUwMkVaOElDWUxJMTYzLm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861512
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9ZQkRRMDhESElEUEEzSjk4MzQ2Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861747
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9ZQ0NKVFQzS1NaUjhNWUpLODI2Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861439
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o?pageToken=CjJjaHVua19kYXRhL25vcm1hbC9uZWdhdGl2ZS9ZR0pTVUFGQ1ZKNTJMSVJEMTQ5Lm5weQ%3D%3D&projection=noAcl&prefix=chunk_data%2Fnormal%2Fnegative HTTP/1.1" 200 861335


In [5]:
model = voxnet.VoxNet(1, "chunk weights")

DEBUG:root:Layer1:Conv3D shape=(None, 32, 14, 14, 14)
DEBUG:root:Layer3:Conv3D shape=(None, 32, 12, 12, 12)
DEBUG:root:Layer4:MaxPool3D shape=(None, 32, 6, 6, 6)
DEBUG:root:Layer5:Flatten shape=(None, 6912)
DEBUG:root:Layer6:Dense shape=(None, 128)
DEBUG:root:Layer8:Dense shape=(None, 1)
INFO:root:Model compiled!


/home/amy/elvo-analysis/ml/models/three_d/voxnet.py:83: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(input_shape=(1, 32, 32..., kernel_size=(5, 5, 5), filters=32, strides=(2, 2, 2), padding="valid", data_format="channels_first", kernel_initializer="normal", kernel_regularizer=<keras.reg..., bias_regularizer=<keras.reg...)`
  b_regularizer=l2(0.001),
/home/amy/elvo-analysis/venv/lib/python3.6/site-packages/keras/activations.py:186: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))
/home/amy/elvo-analysis/ml/models/three_d/voxnet.py:93: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.3)`
  self._mdl.add(Dropout(p=0.3))
/home/amy/elvo-analysis/ml/models/three_d/voxnet.py:105: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(kernel_siz

In [6]:
# Train model on dataset
# model.fit_generator(generator=train_gen.generate(),
#                    validation_data=val_gen.generate(),
#                    steps_per_epoch=training_generator.get_steps_per_epoch(),
#                    validation_steps=validation_generator.get_steps_per_epoch(),
#                    epochs = 15)


model.fit(generator = train_gen.generate(), 
          samples_per_epoch = train_gen.get_steps_per_epoch(),
          nb_epoch = 15, valid_generator = val_gen.generate(), 
          nb_valid_samples = val_gen.get_steps_per_epoch(), 
          verbosity = 1)

INFO:root:Start training


/home/amy/elvo-analysis/ml/models/three_d/voxnet.py:180: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  nb_val_samples=nb_valid_samples,
/home/amy/elvo-analysis/ml/models/three_d/voxnet.py:180: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(generator=<generator..., verbose=1, callbacks=[<keras.ca..., validation_data=<generator..., steps_per_epoch=1208, epochs=15, validation_steps=302)`
  nb_val_samples=nb_valid_samples,


AKZ8T688ZRU9UTY2518_7
Epoch 1/15
DEBUG:urllib3.connectionpool:Resetting dropped connection: www.googleapis.com
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o/chunk_data%2Fnormal%2Fpositive%2FAKZ8T688ZRU9UTY2518_7.npy HTTP/1.1" 200 833
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /download/storage/v1/b/elvos/o/chunk_data%2Fnormal%2Fpositive%2FAKZ8T688ZRU9UTY2518_7.npy?generation=1531836725044852&alt=media HTTP/1.1" 200 65664
(32, 32, 32)
(1, 32, 32, 32)
images shape: 1
BVYSR54MKT5SPMSU310_1
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o/chunk_data%2Fnormal%2Fpositive%2FBVYSR54MKT5SPMSU310_1.npy HTTP/1.1" 200 833
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /download/storage/v1/b/elvos/o/chunk_data%2Fnormal%2Fpositive%2FBVYSR54MKT5SPMSU310_1.npy?generation=1531836792093703&alt=media HTTP/1.1" 200 65664
(32, 32, 32)
(1, 32, 32, 32)
images shape: 2
LE9D7Q73ZSL4N528606_21


(32, 32, 32)
(1, 32, 32, 32)
images shape: 2
ZUEK5YSS7CITVWIP468_3
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o/chunk_data%2Fnormal%2Fpositive%2FZUEK5YSS7CITVWIP468_3.npy HTTP/1.1" 200 833
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /download/storage/v1/b/elvos/o/chunk_data%2Fnormal%2Fpositive%2FZUEK5YSS7CITVWIP468_3.npy?generation=1531838170321072&alt=media HTTP/1.1" 200 65664
(32, 32, 32)
(1, 32, 32, 32)
images shape: 3
GBA7XJ5E7ITWZW7K373
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos/o/chunk_data%2Fnormal%2Fnegative%2FGBA7XJ5E7ITWZW7K373.npy HTTP/1.1" 200 825
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /download/storage/v1/b/elvos/o/chunk_data%2Fnormal%2Fnegative%2FGBA7XJ5E7ITWZW7K373.npy?generation=1531784688353151&alt=media HTTP/1.1" 200 65664
(32, 32, 32)
(1, 32, 32, 32)
images shape: 4
GQNS07SL34K9IUIU212
DEBUG:urllib3.connectionpool:https://www.googleapis.c

ValueError: Input arrays should have the same number of samples as target arrays. Found 13 input samples and 16 target samples.